In [542]:
#user_id                             int64
#email                              object
#first_name                         object
#last_name                          object
#gender                             object
#birthday                   datetime64[ns]
#city                               object
#state                              object
#newsletter_sign_up_date    datetime64[ns]
#order_id                          float64
#purchase_date              datetime64[ns]
#product                            object
#listed_price                      float64
#paid_price                        float64
#age                               float64
#purchase_month               0
#purchase_year                0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import datetime
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
UserOrder = pd.ExcelFile('Patterngonia DB.xlsx')
U = pd.read_excel(UserOrder, 'Users')
O = pd.read_excel(UserOrder, 'Orders')

In [ ]:
cust = pd.merge(O, U, on = 'user_id', how = 'left')

In [ ]:
cust.head()

In [ ]:
print(cust.dtypes)
print(pd.DataFrame([cust.listed_price, cust.paid_price]).transpose().describe())
cust.info

In [ ]:
cust.shape


In [ ]:
print(cust['gender'].value_counts())
print(cust['city'].value_counts())
print(cust['state'].value_counts())
print(cust['product'].value_counts())

In [ ]:
## TO find the age
year = datetime.date.today().year
cust['age'] = year - cust['birthday'].dt.year
cust.head()

In [ ]:
cust["purchase_month"] = cust["purchase_date"].dt.month
cust["purchase_year"] = cust["purchase_date"].dt.year

In [ ]:
# Time period of all purchases

print(cust['purchase_date'].min())
print(cust['purchase_date'].max())

In [ ]:
plt.rcParams["figure.figsize"]=(27,9)
plt.subplot(121)
sns.countplot(cust['gender'])

In [ ]:
# Distplot for age
Age = cust.dropna()
sns.displot(Age.age, kde=False)
plt.show() 

# 1 Categorization by Products - List of users who contribute 58% of the total sales

In [ ]:
#Product wise categorization  
cust.groupby(['product'])['paid_price'].aggregate('sum').reset_index().sort_values('paid_price', ascending = False).head(10)
# Around 58% of the client's income is from Tent product type per year

In [ ]:
# To filter data by Product = 'Tent'
TentCustomers = cust[cust['product'] == 'Tent']
TentCustomers

In [ ]:
#Which month tents are more purchased
TentCustomers.groupby(["purchase_month"])["paid_price"].aggregate('sum').reset_index().sort_values('paid_price', ascending = False)
# 59% of June Month Sales are again from Tent. Which implies more sales are there for tent during the summer holidays

In [ ]:
#Data of all customers who bought tent in June
TentCustomersJune = TentCustomers[TentCustomers["purchase_month"] == 6]
TentCustomersJune
# There are 29 customer

In [ ]:
# User_ID-State-City wise Data of all customers who bought tent in June
TentCustomersJune.groupby(['user_id','state','city'])['paid_price'].agg('sum').reset_index().sort_values('paid_price', ascending = False)


#print(TentCustomers)
#a[a['state'].isin(['Texas','Florida','California','District of Columbia','Pennsylvania'])].sort_values(['state', 'paid_price'], ascending = [True, False])

In [ ]:
TentCustomersJune.groupby(["age_bins"])["paid_price"].aggregate('sum').reset_index().sort_values('paid_price', ascending = False)

# 2. Sending recommendations during the customer’s birthdays

In [ ]:
cust["whoBoughtOnBday"] = (cust["purchase_month"] == (cust['birthday']).dt.month)

In [ ]:
BdayPurchase = cust[cust["whoBoughtOnBday"] == True]
BdayPurchase.shape

In [ ]:
# 10% of income from 184 customers were on their Bday

(BdayPurchase["paid_price"].sum()/cust["paid_price"].sum())*100

# 3. Grouping customers by their purchase patterns based on the purchase month

In [ ]:
# Number of products purchased in a year
# Maximum nunmber of products purchased in which month?

(cust.groupby(["purchase_month"])["paid_price"].agg("sum").reset_index().sort_values("paid_price", ascending = False))


In [ ]:
MonthlySales = cust[cust["purchase_month"].isin([i for i,j in dict((cust.groupby(['purchase_month'])['paid_price'].aggregate('sum'))).items() if i == 6])]
MonthlySales

In [ ]:
MonthlySales.groupby(["product"])["paid_price"].aggregate("sum").reset_index().sort_values("paid_price", ascending=False)
# 59% of June Month Sales are again from Tent. Which implies more sales are there for tent during the summer holidays

# 4. Around 16% of customers are from Texas, Florida & California

In [ ]:
#Highest customers across states

customer_state=cust[['state','user_id']].drop_duplicates()
customer_state.groupby(['state'])['user_id'].aggregate('count').reset_index().sort_values('user_id', ascending=False).head(10)

#Around 16% of cutomers are from Texas, Floria & California

In [ ]:
cust.groupby(['state'])['paid_price'].aggregate('sum').reset_index().sort_values('paid_price', ascending=False)
#163910
#Approx 50 % Sales are from 8 states. Remaining 50% sales are from remaining 42 states. 
#So let us market more and increase the sales in these 8 states

In [ ]:
(cust.groupby(['state'])['paid_price'].aggregate('sum').reset_index().sort_values('paid_price', ascending=False))[:8]

# PCA 

In [ ]:
#PCA
cust_pca = cust

In [ ]:
cust_cat = cust.select_dtypes(include=['object'])
cust_enc = cust_cat.copy()
cust_enc.drop('email', axis=1, inplace=True) 
cust_enc.drop('first_name', axis=1, inplace=True) 
cust_enc.drop('last_name', axis=1, inplace=True) 
#cust_enc.drop('user_id', axis=1, inplace=True) 
#cust_enc.drop('order_id', axis=1, inplace=True) 
print(cust_enc) 

In [ ]:
#ONEHOT ENCODING
cust_enc = pd.get_dummies(cust_enc, columns=['product','gender','city','state'])

In [ ]:
cust_enc = cust_enc.apply(LabelEncoder().fit_transform)

In [ ]:
cust_pca = pd.concat([cust_pca, (cust['purchase_date']).dt.month, (cust['purchase_date']).dt.year], axis = 1)

In [ ]:
cust_pca = cust_pca.drop(['product','gender','city','state'], axis=1)

In [ ]:
cust_pca = pd.concat([cust_pca,cust_enc], axis=1)

In [ ]:
cust_pca.drop(['order_id', 'purchase_date', 'user_id', 'email', 'first_name','last_name', 'birthday', 'newsletter_sign_up_date', 'gender_Female'], axis = 1, inplace = True)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
cust_pca_scale = scale(cust_pca)

In [ ]:
pca = PCA(n_components=44)

In [ ]:
pca.fit(cust_pca_scale)

In [ ]:
var= pca.explained_variance_ratio_

In [ ]:
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

In [ ]:
plt.plot(var1)

In [ ]:
#Looking at above plot I'm taking 30 variables
pca = PCA(n_components=2)
pca.fit(cust_pca_scale)
cust_pca_scale1=pca.fit_transform(cust_pca_scale)

print(cust_pca_scale1)

In [ ]:
temp1=pd.DataFrame(cust_pca_scale1)

In [ ]:
tempU = cust['user_id']
tempO = cust['order_id']

In [ ]:
cust_pca_scale2 = pd.concat([temp1, tempU, tempO], axis = 1)

# Gender wise product Classification

In [ ]:
# Gender wise product classification
ax = sns.countplot(x='product', hue='gender', data = cust)

# There are 73 members of Male Gender more than Female

In [ ]:
#Male leading female by 73#
cust['gender'].value_counts()

# Male Female spending patterns

In [ ]:
cust.groupby(['gender'])['paid_price'].aggregate('sum').reset_index().sort_values('paid_price', ascending = False)
sns.barplot(x='gender', y='paid_price', data=cust) #O/p gives mean

# Both genders contribute equal monetory value to the customer

In [ ]:
MaleFemaleSpendingRatio = cust.groupby(['gender']) 
MaleFemaleSpendingRatio = MaleFemaleSpendingRatio[['paid_price']].sum()
MaleFemaleSpendingRatio.apply(lambda x: x/x.sum()*100).reset_index()

# Creating age bins and classifying the paid price pattern


In [ ]:
cust['age_bins']=pd.cut(cust['age'], [0,15,25,35,45,55,65,75,85,120], labels = ['0-15','15-25','25-35','35-45','45-55','55-65','65-75','75-85','85-120'])
print(cust['age_bins'].head())

In [ ]:
#BarPlot
sns.barplot(x='age_bins', y='paid_price', data = cust)
# 25 to 75 years (i.e) salaried people are buying products

# Average spending - Age wise categorization

In [ ]:
ageSpending = cust.groupby(['age_bins'])
AgeSpending = AgeSpending[['paid_price']].mean()
AgeSpending

# Further drilling down based on genders

In [ ]:
# Genders  in the following age groups spend the following amount for purchase
cust.groupby(['gender', 'age_bins'])['paid_price'].mean()

# Top 10 cities that contribute to the sales of the products

In [ ]:
cust.groupby(['city'])['paid_price'].aggregate('sum').reset_index().sort_values('paid_price', ascending=False).head(10)


# Top 10 city customer base across USA

In [ ]:

customer_city=cust[['city','user_id']]
customer_city.groupby(['city'])['user_id'].aggregate('count').reset_index().sort_values('user_id', ascending=False).head(10)


# Users who have contributed more monitory value to the client

In [ ]:
#Users who are spending maximum with more number of orders
cust.groupby('user_id').agg({'order_id':'count', 'paid_price': 'sum'}).reset_index().sort_values(['paid_price','order_id'], ascending = [False, True]).head(10)